## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

2022-03-18 13:56:30.927624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 13:56:30.927662: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Creating labels

In [2]:
data = []
labels = [] 
classes = 43
curr_path = os.getcwd()

for i in range(classes):
    path = os.path.join(curr_path, 'data/train', str(i))
    images = os.listdir(path)
    
    for a in images:
        try:
            image = Image.open(path + '/' + a)
            image = image.resize((30,30))
            image = np.array(image)
            
            data.append(image)
            labels.append(i)
        except:
            print("Error")

data = np.array(data)
labels = np.array(labels)

In [3]:
data.shape

(39209, 30, 30, 3)

In [4]:
labels.shape

(39209,)

## Splitting the dataset

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(31367, 30, 30, 3) (7842, 30, 30, 3)
(31367,) (7842,)


In [6]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

## Building the model

In [7]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

# compiling
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-03-18 13:56:58.834688: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-18 13:56:58.834753: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (archi-halder): /proc/driver/nvidia/version does not exist
2022-03-18 13:56:58.836090: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 64)          36928     
                                                        

## Training and Validating the model

In [8]:
epochs = 20
history = model.fit(X_train, y_train, batch_size=64,
                    epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/20
491/491 [==============================] - 40s 81ms/step - loss: 1.8170 - accuracy: 0.5510 - val_loss: 0.3574 - val_accuracy: 0.9188
Epoch 2/20
491/491 [==============================] - 42s 86ms/step - loss: 0.5010 - accuracy: 0.8611 - val_loss: 0.1479 - val_accuracy: 0.9612
Epoch 3/20
491/491 [==============================] - 42s 86ms/step - loss: 0.2976 - accuracy: 0.9184 - val_loss: 0.0872 - val_accuracy: 0.9795
Epoch 4/20
491/491 [==============================] - 44s 89ms/step - loss: 0.2536 - accuracy: 0.9309 - val_loss: 0.0554 - val_accuracy: 0.9858
Epoch 5/20
491/491 [==============================] - 44s 89ms/step - loss: 0.1790 - accuracy: 0.9496 - val_loss: 0.0546 - val_accuracy: 0.9853
Epoch 6/20
491/491 [==============================] - 44s 90ms/step - loss: 0.1728 - accuracy: 0.9516 - val_loss: 0.0534 - val_accuracy: 0.9855
Epoch 7/20
491/491 [==============================] - 47s 96ms/step - loss: 0.1573 - accuracy: 0.9569 - val_loss: 0.0437 - val_accuracy:

## Saving the trained model

In [9]:
model.save('heavy_model.h5')

## Loading the model

In [9]:
from tensorflow.keras.models import load_model

saved_model = load_model('heavy_model.h5')

## Plotting the accuracy curve 

In [10]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>

## Testing

In [11]:
import pandas as pd

y_test = pd.read_csv('data/Test.csv')

labels = y_test['ClassId'].values
imgs = y_test['Path'].values

data = []

for i in imgs:
    image = Image.open('data/' + i)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test = np.array(data)

pred = model.predict(X_test)

## Accuracy Score

In [30]:
pred.shape

(12630, 43)

In [31]:
labels.shape

(12630,)